# Writing out a USGSCSM ISD from a PDS3 Messenger MDIS image

In [25]:
import ale
from ale.drivers.messenger_drivers import MessengerMdisPds3NaifSpiceDriver
from ale.formatters.usgscsm_formatter import to_usgscsm
import json
import os

## Instantiating an ALE driver

ALE drivers are objects that define how to acquire common ISD keys from an input image format, in this case we are reading in a PDS3 image using NAIF SPICE kernels for exterior orientation data. If the driver utilizes NAIF SPICE kernels, it is implemented as a [context manager](https://docs.python.org/3/reference/datamodel.html#context-managers) and will furnish metakernels when entering the context (i.e. when entering the `with` block) and free the metakernels on exit. This maintains the integrity of spicelib's internal data structures. These driver objects are short-lived and are input to a formatter function that consumes the API to create a serializable file format. `ale.formatters` contains available formatter functions. 

The default config file is located at `ale/config.yml` and is copied into your home directory at `.ale/config.yml` on first use of the library. The config file can be modified using a text editor. `ale.config` is loaded into memory as a dictionary. It is used to find metakernels for different missions. For example, there is an entry for LRO that points to `/usgs/cpkgs/isis3/data/lro/kernels/mk/` by default. If you want to use your own metakernels, you will need to udpate this path. For example, if the metakernels are located in `/data/lrolrocnac/mk/` the LRO entry should be updated with this path. If you are using the default metakernels, then you do not need to update the path.

ALE has a two step process for writing out an ISD: 1. Instantiate your driver (in this case `MessengerMdisPds3NaifSpiceDriver`) within a context and 2. pass the driver object into a formatter (in this case, `to_usgscsm`).  

Requirements:
 * A PDS3 Messenger MDIS image
 * NAIF metakernels installed
 * Config file path for LRO (ale.config.lro) pointing to LRO NAIF metakernel directory 
 * A conda environment with ALE installed into it usisng the `conda install` command or created using the environment.yml file at the base of ALE.

In [26]:
# printing config displays the yaml formatted string
print(ale.config)

# config object is a dictionary so it has the same access patterns 
print('MDIS spice directory:', ale.config['mdis'])

# updating config for new MDIS path in this notebook 
# Note: this will not change the path in `.ale/config.yml`. This change only lives in the notebook.
ale.config['mdis'] = '/work/users/sgstapleton/ale/'

cassini: /usgs/cpkgs/isis3/data/cassini/kernels/mk/
dawn: /data/spice/dawn-m_a-spice-6-v1.0/dawnsp_1000/extras/mk
kaguya: /data/spice/SELENE/kernels/mk/
lro: /scratch/jlaura/spice/lro-l-spice-6-v1.0/lrosp_1000/extras/mk/
mdis: /data/spice/mess-e_v_h-spice-6-v1.0/messsp_1000/extras/mk
mro: /data/spice/mro-m-spice-6-v1.0/mrosp_1000/extras/mk
spice_root: /data/spice/

MDIS spice directory: /data/spice/mess-e_v_h-spice-6-v1.0/messsp_1000/extras/mk


In [27]:
# change to desired PDS3 image path 
fileName = '/home/kdlee/builds/ale/EN1072174528M.IMG'

# metakernels are furnsh-ed when entering the context (with block) with a driver instance
# most driver constructors simply accept an image path 
with MessengerMdisPds3NaifSpiceDriver(fileName) as driver:
    # pass driver instance into formatter function
    usgscsmString = to_usgscsm(driver)

### Write ISD to disk 

ALE formatter functions generally return bytes or a string that can be written out to disk. ALE's USGSCSM formatter function returns a JSON encoded string that can be written out using any JSON library. 

USGSCSM requires the ISD to be colocated with the image file with a `.json` extension in place of the image extension.

In [28]:
# load the json encoded string ISD
usgscsm_dict = json.loads(usgscsmString)

# strip the image file extension and append .json 
jsonFile = os.path.splitext(fileName)[0] + '.json'

# write to disk 
with open(jsonFile, 'w') as fp:
    json.dump(usgscsm_dict, fp)

In [31]:
import knoten
import csmapi
model="USGS_ASTRO_LINE_SCANNER_SENSOR_MODEL"
plugin = csmapi.Plugin.getList()[0]
isd = csmapi.Isd('jsonFile')
warns = csmapi.WarningList()
if plugin.canModelBeConstructedFromISD(isd, model, warns):
    print("GOT HERE")
    camera = plugin.constructModelFromISD(isd, model)
    print(camera)
else:
    print("CAN'T CONSTRUCT")
    for item in warns:
        print(item.getMessage())

CAN'T CONSTRUCT


In [3]:
from pysis import isis

In [18]:
output = isis.campt(from_='/home/kdlee/builds/ale/EN1072174528M.cub')

In [13]:
import pvl
import numpy as np
pvl_output = pvl.loads(output)

In [14]:
bodyfixed = pvl_output['GroundPoint']['BodyFixedCoordinate']
bodyfixed = np.asarray(bodyfixed.value) * 1000

In [23]:
from knoten import csm
csm.generate_ground_point(dem=0, image_pt=(256, 256), camera=camera)

NameError: name 'camera' is not defined

In [19]:
output

b'Group = GroundPoint\n  Filename                   = /home/kdlee/builds/ale/EN1072174528M.cub\n  Sample                     = 256.0\n  Line                       = 256.0\n  PixelValue                 = 284\n  RightAscension             = 166.36587753487 <DEGREE>\n  Declination                = -43.071551129512 <DEGREE>\n  PlanetocentricLatitude     = 46.273591918608 <DEGREE>\n  PlanetographicLatitude     = 46.273591918608 <DEGREE>\n  PositiveEast360Longitude   = 248.06178671888 <DEGREE>\n  PositiveEast180Longitude   = -111.93821328112 <DEGREE>\n  PositiveWest360Longitude   = 111.93821328112 <DEGREE>\n  PositiveWest180Longitude   = 111.93821328112 <DEGREE>\n  BodyFixedCoordinate        = (-629.77103851644, -1563.590514698,\n                                1762.3084455539) <km>\n  LocalRadius                = 2438679.5434985 <meters>\n  SampleResolution           = 1.4794782052815 <meters/pixel>\n  LineResolution             = 1.4794782052815 <meters/pixel>\n  ObliqueDetectorResolution 